In [29]:
import pandas as pd
import re
from datetime import datetime
import pytz
from urllib.parse import urlparse
from urllib.parse import parse_qs
import numpy as np 

In [30]:
# datetime.datetime.strptime('13/Nov/2015:11:45:42','%Y-%m-%d:%H:%M:%S +%f')

In [31]:
import os
current_file = os.getcwd()
myfile = os.path.join(current_file, 'www.pangaea.de-access-anon.log.bz2')

In [32]:
#ref: https://mmas.github.io/read-apache-access-log-pandas
logData = pd.read_csv(myfile,
    sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
    engine='python',
    na_values='-',
    header=None,
    usecols=[0, 3, 4, 5, 7, 8],encoding="utf-8",
    names=['ip', 'time', 'request', 'status', 'referer', 'user_agent'])

In [33]:
logData.head(1)

,ip,time,request,status,referer,user_agent
0,68aa873d61a6586244b0892d7bca2573,[30/Sep/2019:06:25:44 +0000],"""GET /search?ie=UTF-8&q=parameter%3A%22Bathysi...",301.0,"""-""","""Mozilla/5.0 (compatible; AhrefsBot/6.1; +http..."


In [34]:
logData['status'] = logData['status'].fillna(0) #convert non-finite values (NA or inf) to integer
logData['status'] = logData['status'].astype(int)
logData.head(1)

,ip,time,request,status,referer,user_agent
0,68aa873d61a6586244b0892d7bca2573,[30/Sep/2019:06:25:44 +0000],"""GET /search?ie=UTF-8&q=parameter%3A%22Bathysi...",301,"""-""","""Mozilla/5.0 (compatible; AhrefsBot/6.1; +http..."


In [35]:
logData['request']= logData['request'].apply(lambda x: re.sub(r'^"|"$', '', str(x)) )
logData['user_agent']= logData['user_agent'].apply(lambda x: re.sub(r'^"|"$', '',str(x)) )
logData['referer']= logData['referer'].apply(lambda x: re.sub(r'^"|"$', '',str(x)) )

In [36]:
logData.shape

(3874088, 6)

In [37]:
logData['request_type'] = logData['request'].str.split().str[0]
logData['resource'] = logData['request'].str.split().str[1]
logData.head(1)

,ip,time,request,status,referer,user_agent,request_type,resource
0,68aa873d61a6586244b0892d7bca2573,[30/Sep/2019:06:25:44 +0000],GET /search?ie=UTF-8&q=parameter%3A%22Bathysip...,301,-,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...,GET,/search?ie=UTF-8&q=parameter%3A%22Bathysiphon+...


In [38]:
logData = logData[(logData.request_type == 'GET') & (logData.status == 200)]
logData.shape

(3063078, 8)

In [39]:
logData=logData.drop('request', axis=1)
logData.head(1)

,ip,time,status,referer,user_agent,request_type,resource
3,75892c86ec3272b7fa79e0f2c375a9f5,[30/Sep/2019:06:25:56 +0000],200,-,Mozilla/5.0 (compatible; SemrushBot/6~bl; +htt...,GET,/?maxdate=2005-12-31T23%3A59%3A59&mindate=2005...


In [40]:
logData = logData[~logData['resource'].str.match(
    r'^/media|^/static|^/admin|^/robots.txt$|^/favicon.ico$')]
logData = logData[~logData['user_agent'].str.match(
    r'.*?bot|.*?spider|.*?crawler|.*?slurp', flags=re.I).fillna(False)]
logData.shape

(1858386, 7)

In [41]:
logData.head(1)

,ip,time,status,referer,user_agent,request_type,resource
9,eb6f30241bf7ff9bf1c6ddb6d5ea2d66,[30/Sep/2019:06:26:12 +0000],200,https://doi.pangaea.de/10.1594/PANGAEA.890974,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6...,GET,/?q=SOCATv5


In [42]:
#https://doi.pangaea.de, https://www.pangaea.de
#use referrel to filter out external requests - only request originated from the portal will be considered
logData = logData[logData.referer.str.startswith('https://doi.pangaea.de') | logData.referer.str.startswith('https://www.pangaea.de')]
logData.shape

(1335062, 7)

In [43]:
#exclude queries originated from external search -> /advanced/search.php, /advanced/warehouse.php
logData = logData[~logData.resource.str.contains('/advanced/')]
logData.shape

(1234130, 7)

In [44]:
regex = '(\?q\=)(.*?)\&'
def parse_query(url):
    parsed = urlparse(url)
    l = None
    if 'q' in parse_qs(parsed.query):
        l = parse_qs(parsed.query)['q']
        l= l[0].lower()
    else:
        search = re.search(regex, url, re.IGNORECASE)
        if search:
            l= search.group(2).lower()
    return l

In [45]:
searchData =logData.copy()
searchData.shape

(1234130, 7)

In [46]:
searchData= searchData[searchData.resource.str.contains('q=')] 
searchData.shape
# searchData.to_csv('searchData')

(5952, 7)

In [47]:
keyWords = searchData['resource']
# keyWords.value_counts().head(50)
# keyWords.to_csv('search1')

In [48]:
keyWords = searchData['resource'].apply(parse_query)
keyWords.head(5)

9                        socatv5
775     parameter:"file content"
2159       project:label:sponges
4107        parameter:"salinity"
4244        event:label:lavpicco
Name: resource, dtype: object

In [49]:
top50KeyWords = keyWords.value_counts().head(50)
# top50KeyWords

In [50]:
df = pd.DataFrame({'word':top50KeyWords.index, 'count':top50KeyWords.values})

In [51]:
def checkPar(x):
    if (x == True):
        return 'parameter'
    else: 
        return 'unknown'

In [52]:
df['type']= df['word'].str.contains('parameter')
df['type'] = df['type'].apply(lambda x: checkPar(x))

In [53]:
df.loc[4, 'type'] = 'author'
df.loc[13, 'type'] = 'cruise'
df.loc[14, 'type'] = 'isotope oxygen-18'
df.loc[15, 'type'] = 'paper'
df.loc[18, 'type'] = 'drilling site'
df.loc[20, 'type'] = 'cruise'
df.loc[24, 'type'] = 'location'
df.loc[25, 'type'] = 'sediment'
df.loc[26, 'type'] = 'igkb'
df.loc[30, 'type'] = 'marine site'
df.loc[31, 'type'] = 'cruise'
df.loc[34, 'type'] = 'sediment'
df.loc[35, 'type'] = 'magnetic field'
df.loc[36, 'type'] = 'japan current'
df.loc[39, 'type'] = 'sea surface temprature'
df.loc[41, 'type'] = 'paper'
df.loc[43, 'type'] = 'author'
df.loc[45, 'type'] = 'station'

In [54]:
#remove 'parameter:' from the word column send excel sheet
# add url next to type
df['word'] = df['word'].str.replace('parameter:', '')
df['word'] = df['word'].str.replace('"', '')

In [62]:
types = df['type'].value_counts()
barData = pd.DataFrame({'type':types.index, 'count':types.values})
barData['count']=barData['count'].astype(int)
barData

,type,count
0,parameter,21
1,unknown,11
2,cruise,3
3,author,2
4,paper,2
5,sediment,2
6,location,1
7,japan current,1
8,station,1
9,magnetic field,1


In [63]:
import plotly.express as px
fig = px.bar(barData, x='type', y='count')
fig.show()

In [64]:
import plotly.express as px
fig = px.pie(barData, values='count', names='type')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()